In [24]:
# Penguin loves chipmunk so, so, so much

In [25]:
#Chipmunk loves her penguin so so so much too

In [8]:
print("Chipmunk penguin and bob")

Chipmunk penguin and bob


In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import re
import time

# import packages
from selenium import webdriver
import time

# for whatever reason, selenium gets to the end of a page, and keeps trying to click buttons
# need to try except this error away
from selenium.common.exceptions import StaleElementReferenceException

df = pd.DataFrame(columns=['title', 'genre', 'language', 'runtime', 'release_date', 'description', 'rating', 
                           'box_office', 'tomatometer', 'audience_score', 'tomato_score_count', 'audience_score_count',
                          'reviews', 'img_url'])

In [11]:
# get_links gets all links for movies of genre "genre" with score between "score_min" and "score_max"
# input "score_min", "score_max", and "genre" as integers
def get_links(score_min, score_max, genre):

    # create url to scrape links from
    url = "https://www.rottentomatoes.com/browse/dvd-streaming-all?" + \
        "minTomato=" + str(score_min) + "&maxTomato=" + str(score_max) + \
        "&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=" + \
        str(genre) + "&sortBy=release"

    # initialize empty list to put URLs in
    endings = []
    
    # open a chrome webbrowser, and go to the URL
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--remote-debugging-port=9222")
    driver = webdriver.Chrome(chrome_options=chromeOptions)
    driver.get(url)
    
    # look for button to load all the movies
    buttons = driver.find_elements_by_class_name('btn.btn-secondary-rt.mb-load-btn')
    
    # wait five seconds for the page to load
    time.sleep(5)
    
    # while there are buttons to click, keep clicking them
    while(buttons):
        try:
            buttons[0].click()
            
            # look for more buttons to click
            buttons = driver.find_elements_by_class_name('btn.btn-secondary-rt.mb-load-btn')
            time.sleep(2)
        except StaleElementReferenceException:
            # once we reach bottom of page, break
            print("Reached bottom of page, scraping links")
            break
    
    # look for the boxes containing info about the movies
    infos = driver.find_elements_by_class_name('movie_info')
    for info in infos:
        # convert to beautiful soup objects
        soup = BeautifulSoup(info.get_attribute('innerHTML'), "html.parser")
        for link in soup.findAll('a'):
            # extract hrefs
            endings.append(link.get('href'))
    
    # close the browser once done
    driver.quit()
    
    return endings

In [19]:
def title(soup):
    try:
        return soup.find("h1").text
    except:
        return None

def genre(soup):
    try:
        return " ".join(soup.find("div", {"class": "meta-value genre"}).text.split())
    except:
        return None

def box_office(box_office_string):
    new_string = box_office_string.strip("$")
    if (new_string[len(new_string)-1] == "K"):
        return int(float(new_string.strip("K")) * 1000)
    elif (new_string[len(new_string)-1] == "M"):
        return int(float(new_string.strip("M")) * 1000000)
    elif (new_string[len(new_string)-1] == "B"):
        return int(float(new_string.strip("B")) * 1000000000)
    else:
        return new_string
    
def runtime(runtime_strings):
    try:
        runtime_in_minutes = 60 * int(re.sub("h", "", runtime_strings[0])) + int(re.sub("m", "", runtime_strings[1]))
    except:
        try:
            runtime_in_minutes = 60 * int(re.sub("h", "", runtime_strings[0]))
        except:
            runtime_in_minutes = int(re.sub("m", "", runtime_strings[0]))
    return runtime_in_minutes
    
#language, runtime, release_date, rating, box_office
def get_attributes(soup):
    attributes = {'language':None, 'runtime':None, 'release_date':None, 'rating':None, 'box_office':None}
    for item in soup.find_all("li", {"data-qa":"movie-info-item"}):
        key = item.find('div', {'class':'meta-label subtle'}).text.strip()
        if key == "Original Language:":
            attributes['language'] = item.find('div', {'class':'meta-value'}).text.strip()
        elif key == "Release Date (Streaming):":
            attributes['release_date'] = pd.to_datetime(item.find('div', {'class':'meta-value'}).text.strip()).date()
        elif key == "Box Office (Gross USA):":
            attributes['box_office'] = box_office(item.find('div', {'class':'meta-value'}).text.strip())
        elif key == "Runtime:":
            runtime_text = item.find('div', {'class':'meta-value'}).text.strip().split()
            attributes['runtime'] = runtime(runtime_text)
        elif key == "Rating:":
            attributes['rating'] = re.sub("\([^()]*\)", "", item.find('div', {'class':'meta-value'}).text.strip())
    return attributes

def image(soup):
    try:
        return soup.find('img', {'class':'posterImage js-lazyLoad'}).get('data-src')
    except:
        return None

def description(soup):
    try:
        return soup.find("div", {"id":"movieSynopsis"}).text.strip()
    except:
        return None

def tomatometer(soup):
    try:
        return int(soup.find("span", {"class":"mop-ratings-wrap__percentage"}).text.strip().strip("%"))
    except:
        return None

def audience_score(soup):
    try:
        return int(soup.find_all("span", {"class":"mop-ratings-wrap__percentage"})[1].text.strip().strip("%"))
    except:
        return None
    
def tomato_score_count(soup):
    try:
        return int(soup.find("small", {"class":"mop-ratings-wrap__text--small"}).text.strip())
    except:
        return None

def audience_score_count(soup):
    try:
        return int(re.sub("[^0-9]", "", soup.find_all("strong", {"class":"mop-ratings-wrap__text--small"})[1].text.strip()))
    except:
        return None

In [13]:
def get_reviews(soup):
    return [x.text.strip() for x in soup.find_all('div', {'class':'the_review'})]

def scrape_reviews(url):
    reviews = []
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    #finding how many pages there are
    try:
        pageInfo = soup.find("span", {"class":"pageInfo"}).text.split()
        pages_count = int(pageInfo[len(pageInfo)-1])
    except:
        pages_count=1
    #first page
    reviews.extend(get_reviews(soup))
    
    #the rest of the pages
    if pages_count > 1:
        for i in range(2, pages_count+1):
            r_ = requests.get(url+'?type=&sort=&page='+str(i))
            soup_ = BeautifulSoup(r_.content)
            reviews.extend(get_reviews(soup_))
    return reviews

In [14]:
#function to extract all information from a specific movie's url and then add it to the dataframe
def scrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    
    attributes = get_attributes(soup)
    
    row = {'title':title(soup), 'genre':genre(soup), 'language':attributes.get('language'), 'runtime':attributes.get('runtime'), 'release_date':attributes.get('release_date'),
          'description':description(soup), 'rating':attributes.get('rating'), 'box_office':attributes.get('box_office'), 'tomato_score_count':tomato_score_count(soup),
          'audience_score_count':audience_score_count(soup), 'tomatometer':tomatometer(soup), 'audience_score':audience_score(soup),
          'reviews':scrape_reviews(url+'/reviews'), 'img_url':image(soup)}
    return row

In [15]:
def batch(score_min, score_max, genre, storage_file):
    endings = get_links(score_min, score_max, genre)
    df = pd.DataFrame(columns=['title', 'genre', 'language', 'runtime', 'release_date', 'description', 'rating', 
                           'box_office', 'tomatometer', 'audience_score', 'tomato_score_count', 'audience_score_count',
                          'reviews', 'img_url'])
    for ending in endings:
        url = 'https://www.rottentomatoes.com' + ending
        try:
            df.loc[len(df)] = scrape(url)
        except:
            print(url + ' didn\'t work')
            
    df.to_pickle(storage_file)

In [23]:
def batch_2(urls_list, storage_file):
    for url in urls_list:
        try:
            df.loc[len(df)] = scrape(url)
        except:
            print(url + ' didn\'t work')
            
    df.to_pickle(storage_file)

In [16]:
batch(96, 100, 1, 'df1.pkl')

/home/agallier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


Reached bottom of page, scraping links
https://www.rottentomatoes.com/m/mad_max_fury_roaddidn't work
https://www.rottentomatoes.com/m/kill_zone_2_2016didn't work
https://www.rottentomatoes.com/m/sherlock-jrdidn't work
https://www.rottentomatoes.com/m/a-bittersweet-life-dalkomhan-insaengdidn't work
https://www.rottentomatoes.com/m/touch_of_zendidn't work
https://www.rottentomatoes.com/m/1054794-three_musketeersdidn't work
https://www.rottentomatoes.com/m/crouching_tiger_hidden_dragondidn't work
https://www.rottentomatoes.com/m/stalag_17didn't work


In [17]:
pickle_df = pd.read_pickle('df1.pkl')
pickle_df

,title,genre,language,runtime,release_date,description,rating,box_office,tomatometer,audience_score,tomato_score_count,audience_score_count,reviews,img_url
0,Mind Game,"comedy, mystery and thriller, adventure",Japanese,103,2018-08-28,Nishi travels to heaven and back.,None,5600,100,93,11,3388,"[...no end of phantasmagorical weirdness..., C...",https://resizing.flixster.com/pbsRZk9L4C3DbP6g...
1,Speedy,comedy,None,87,2016-08-10,"Chronically unemployed Yankees fan Harold ""Spe...",None,None,100,89,6,708,"[Filmed on the streets of New York in 1928, Sp...",https://resizing.flixster.com/BmfEh1bo3XW2pAi5...
2,Zootopia,"kids and family, animation, adventure, comedy",English,108,2016-06-07,From the largest elephant to the smallest shre...,PG,None,98,92,293,101571,[The abundance of humor and action help to mas...,https://resizing.flixster.com/7wYAhoui16w8hl1n...
3,'71,"mystery and thriller, action",English,99,2016-03-28,A young British soldier (Jack O'Connell) must ...,R,1300000,96,81,138,17433,['71 immerses the viewer in a brutal world whe...,https://resizing.flixster.com/UjH7nm99uLHCPg7k...
4,Theeb,"mystery and thriller, adventure, drama",Arabic,100,2016-05-28,"In 1916, Theeb lives with his Bedouin tribe in...",None,277600,97,76,72,5578,"[For all its beauty, the stark simplicity of t...",https://resizing.flixster.com/SekvcsohG_UmEebf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Murder at the Gallop,"mystery and thriller, crime, drama",English,81,2009-01-01,"While out collecting for charity, Miss Jane Ma...",None,None,100,73,6,398,[The formidable jowls and unconquerable tweedi...,https://resizing.flixster.com/lqzw0bHT4jb_bb39...
200,Don Juan,adventure,None,111,2012-04-16,The famous playboy Don Juan (John Barrymore) g...,None,None,100,60,5,156,[The first in which music and sound effects we...,https://resizing.flixster.com/26IWTK7O7UaCTYV1...
201,Tarzan and His Mate,adventure,English,93,2017-04-05,Englishman Harry Holt (Neil Hamilton) joins a ...,None,None,100,75,17,468,[The current Tarzan is even better than its pr...,https://resizing.flixster.com/JmgMxdKPlStpe-vo...
202,Emil and the Detectives,"kids and family, adventure",English,99,2016-06-17,A 12-year-old (Bryan Russell) employs a group ...,None,None,100,53,5,27,[[Director Peter Tweksbury] has kept the kiddi...,https://resizing.flixster.com/BxbPiYAN9ISFK2Fh...


In [24]:
try_again = ['https://www.rottentomatoes.com/m/mad_max_fury_road',
'https://www.rottentomatoes.com/m/kill_zone_2_2016',
'https://www.rottentomatoes.com/m/sherlock-jr',
'https://www.rottentomatoes.com/m/a-bittersweet-life-dalkomhan-insaeng',
'https://www.rottentomatoes.com/m/touch_of_zen',
'https://www.rottentomatoes.com/m/1054794-three_musketeers',
'https://www.rottentomatoes.com/m/crouching_tiger_hidden_dragon',
'https://www.rottentomatoes.com/m/stalag_17']

batch_2(try_again, 'df2.pkl')

In [42]:
CURRENT_LOWER_BOUND = 0
CURRENT_UPPER_BOUND = 5
CURRENT_PICKLE_FILE = 3
CURRENT_GENRE_INDEX = 0
genres = [1, 2, 4, 5, 6, 8, 9, 10, 11, 13, 18, 14]

In [43]:
def auto_batch(num_intervals):
    global CURRENT_LOWER_BOUND, CURRENT_UPPER_BOUND, CURRENT_PICKLE_FILE, CURRENT_GENRE_INDEX
    for i in range(num_intervals):
        try:
            batch(CURRENT_LOWER_BOUND, CURRENT_UPPER_BOUND, genres[CURRENT_GENRE_INDEX], 'df'+str(CURRENT_PICKLE_FILE)+'.pkl')
        except:
            print("Batch", i, "failed")
            print(CURRENT_LOWER_BOUND, CURRENT_UPPER_BOUND, CURRENT_PICKLE_FILE, CURRENT_GENRE_INDEX)
        if CURRENT_LOWER_BOUND == 0:
            CURRENT_LOWER_BOUND = 6
        else:
            CURRENT_LOWER_BOUND += 5
        CURRENT_UPPER_BOUND += 5
        CURRENT_PICKLE_FILE += 1
        if CURRENT_UPPER_BOUND > 100:
            CURRENT_LOWER_BOUND = 0
            CURRENT_UPPER_BOUND = 6
            CURRENT_GENRE_INDEX += 1
        if CURRENT_GENRE_INDEX > 11:
            print("Bob is done")
            return 0
        print("Batch", i, "is done")
    print("End of batch:", CURRENT_LOWER_BOUND, CURRENT_UPPER_BOUND, CURRENT_PICKLE_FILE, CURRENT_GENRE_INDEX)

In [47]:
auto_batch(1)

/home/agallier/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use options instead of chrome_options


Reached bottom of page, scraping links
Batch 0 is done
End of batch: 11 15 5 0


6